## 背景

在订单投放前或投放时，运营会询量，获取数据的信息主要有来源有两个：

1. easyreport报表查询。
    + [媒体快速估量](http://easyreport.fancydsp.com/projects/24/dashboards/284/preview/)可以查看媒体真实发量请求数，UV数，及竞价曝光率。
    + [媒体广播量报表](http://easyreport.fancydsp.com/projects/24/dashboards/274/preview/)可以查看媒体广播量以及更细的参竞情况，无法查看UV数信息。
2. 写sql在离线表查询
   大部分情况需要使用离线表查询日志信息，表的使用可参考[基础数据篇]



## 例子
### 需求
逻辑1、独立点击，不管一个用户点击多少次，都算一次，把这一次放在最后一次曝光里面。比如一个用户看了5次广告，其中1、2、3次点击过广告，只在频次5里面的独立点击+1，其他都算0  

逻辑2、点击，一个用户看了5次广告，其中1、2、3次点击过广告，请在曝光频次5的点击数据里面+3，其他频次的点击数据都算0  
投放时间：3.5-3.14  

### 操作
### 首先根据需求写sql 确认逻辑 （和运营确认逻辑相同）

In [1]:
# 准备工作
import numpy  as np
import pandas as pd
#连接presto库
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *

# 本地则使用代理，默认socks5端口号为18888
import os
if os.environ.get('JUPYTERENV') != 'release':
    import socks
    import socket
    socks.set_default_proxy(socks.SOCKS5, "localhost", 18888)
    socket.socket = socks.socksocket

presto_engine = create_engine('presto://192.168.28.111:10080/hive/dwd')

In [7]:
#根据需求在离线表中查询数据
xl_df = pd.read_sql('''
select a.imp as "频次", a.area_name as "城市"
    , count(distinct a.uid) as "曝光uv"
    , count(distinct b.uid) as "独立点击"
    , sum(b.clk) as "点击"
from (
    select if(ext_user_id <> '', ext_user_id, concat(ip, user_agent)) uid
         , area_name
         , count(distinct ext_request_id)                             imp
    from dwd.d_ad_impression a
    left join dim.dim_area b
    on cast(a.ext_geo_id as varchar) = b.geo_b_id
    where ext_adgroup_id in
          (251256, 251257, 251258, 251259, 251260, 251261, 251262, 251263, 251264, 251265, 251266, 251267, 251268,
           251269, 251270, 251271, 251272, 251273, 251274, 251275, 251276, 251277, 251278, 251279, 251280, 251281,
           251282, 251283, 251284, 251285, 251286, 251287, 251288, 251289, 251290, 251291, 251292, 251293, 251294,
           251295, 251296, 251297, 251321, 251322, 251323, 251324, 251325, 251326)
      and thisdate >= '2021-03-05'
      and thisdate <= '2021-03-14'
    group by 1, 2
) a
left join (
    select if(ext_user_id <> '', ext_user_id, concat(ip, user_agent)) uid
         , area_name
         , count(distinct ext_request_id)                             clk
    from dwd.d_ad_clicklog_day a
    left join dim.dim_area b on cast(a.ext_geo_id as varchar) = b.geo_b_id
    where ext_adgroup_id in
          (251256, 251257, 251258, 251259, 251260, 251261, 251262, 251263, 251264, 251265, 251266, 251267, 251268,
           251269, 251270, 251271, 251272, 251273, 251274, 251275, 251276, 251277, 251278, 251279, 251280, 251281,
           251282, 251283, 251284, 251285, 251286, 251287, 251288, 251289, 251290, 251291, 251292, 251293, 251294,
           251295, 251296, 251297, 251321, 251322, 251323, 251324, 251325, 251326)
      and thisdate >= '2021-03-05'
      and thisdate <= '2021-03-14'
      and ignore_rule = 0
    group by 1, 2
) b
on a.uid = b.uid and a.area_name = b.area_name
group by 1, 2
''', presto_engine)
xl_df

,频次,城市,曝光uv,独立点击,点击
0,8,深圳,10,1,1
1,16,深圳,1,1,1
2,8,上海,5,1,1
3,7,北京,202,18,20
4,3,上海,19870,997,1057
5,1,深圳,159911,3758,3758
6,3,深圳,11505,669,712
7,3,北京,17389,929,998
8,5,深圳,5711,512,580
9,7,深圳,207,20,24


### 将数据导出到excel文件
### 检查数据有无异常 
是否有点击>曝光，点击uv>点击 类似异常情况  
如果有 修改sql重新出数据
### 将数据反馈给运营 进行二次检查